### Telegram Scratch

In [18]:
import numpy as np
import pandas as pd
from telethon import TelegramClient
import os
import re
import requests
import json
import asyncio

In [2]:
base_path = os.path.dirname(os.getcwd())
thr_int_path = os.path.join(base_path, 'Theat_intel')
ref_path = os.path.join(thr_int_path, 'Reference')
output_path = os.path.join(thr_int_path, 'Output')

In [10]:
os.chdir(ref_path)
# import scraping functions
import scrape
tel_tags = pd.read_csv('telegram_tags.csv')
# grab app_id and hash
api_id = int(tel_tags[tel_tags['Type']=='app_id']['Value'].iloc[0])
api_hash = tel_tags[tel_tags['Type']=='hash']['Value'].iloc[0]

In [11]:
client=TelegramClient('anon', api_id, api_hash)

### Test Usage

**<font color=cyan>Send test message to own channel</font>**

In [8]:
async def main():
    async with client: 
        await client.send_message('me','Hello from Python!')
# with client:
#     client.loop.run_until_complete(main())
await main()

In [25]:
# try to join a channel
from telethon.tl.functions.channels import JoinChannelRequest
async def join_channel(client,channel_link):
    try:
        if not client.is_connected():
            await client.connect()

        await client(JoinChannelRequest(channel_link))
        print(f'Succesfully joined channel {channel_link}')

    except Exception as e:
        print(f'Failed to join channel {channel_link}', e)

async def main():
    async with client:
        channel_link = 'https://t.me/BinancePumpTracker'
        await join_channel(client, channel_link) 

await main()

Succesfully joined channel https://t.me/BinancePumpTracker


### Scrape Messages

In [25]:
num_msgs = 5

async def scrape_message(client, channel, limit=100):
    message_list = []
    try:
        if not client.is_connected():
            await client.connect()  # Ensure connection before making requests
        
        async for each_message in client.iter_messages(channel, limit=limit):
            if each_message.text:  # Check if the message has text content
                message_list.append(each_message.stringify())
                # print(each_message.stringify())  # Print the message content
    except Exception as e: 
        print(f'Failed connection to channel {channel}. Error: {e}')
    return message_list

# async def main():
async with client:  # Ensure the client is properly connected
    # channel_link = 'https://t.me/BinancePumpTracker'
    channel_link = 'BinancePumpTracker'  # Use the channel username, NOT the full URL!
    message_list = await scrape_message(client, channel_link, 5) 

# async def main():
    # async with client:  # Ensure the client is properly connected
    #     # channel_link = 'https://t.me/BinancePumpTracker'
    #     channel_link = 'BinancePumpTracker'  # Use the channel username, NOT the full URL!
    #     message_list = await scrape_message(client, channel_link, 5) 


In [48]:
msg_list_proc = []
# regex_start = r"message='"
regex_start = r'message=(["\'])(.*?)\1' # captures either single or double quote
regex_end = r"out=False"
re_cut = r'\\(n|t|xa0|\\)'

for i in range(num_msgs):
    print(i)
    msg_str = message_list[i]
    msg_str_stage1 = scrape.pullsubstr_begend(regex_start, regex_end, msg_str)
    msg_str_stage2 = re.sub(re_cut, '', msg_str_stage1)
    msg_list_proc.append(msg_str_stage2)

0
1
2
3
4


In [49]:
for i in range(num_msgs):
    print(msg_list_proc[i])

message='#PIVX 👈 https://www.binance.com/en/trade/PIVX_BTCBuying Zone 310-330 Sell🤑 360🤑 360- 400🚀 400- 450 💃 450 -500 & aboveBullish above- 300🔼Death Zone below-300🔽',
	
message='#ADX 👈https://www.binance.com/en/trade/ADX_BTCBuying Zone 230-250 👆Sell🤑 270-280🤑 280-310🚀 310-340💃 340-370 😱 370- 400 & aboveBullish over - 220',
	
message="#ADX 👈 ✨Call given hereHit 305Targets 1 to 5✅64% safe profit If you invested 1 btc it's now 1.64 btc now 🤑🤑✨Signal before pump @ low levelAlways trade with us for max profit 😊🤑💃We are best pump tracker 🤑Consider your profit to join us in VIP🟩🟩🟩🟩🟩🟩🟩🟩",
	
message='🔼 🥰😇🥰🥰 hits an Impressive 202.57% APY! 👀📣CoinEx’s latest empowerment program is setting new records—this is the highest APY in 2 years! ▶️ Don’t miss CoinEx’s 6th Launchpool Event: 🎆Start with just 100 $CET 🟥Dive into a massive 13,000,000 $HOLD pool 💰 ✅Estimated APY could soar as high as 103%! 😱 ▶️Claim your $CET now: https://www.coinex.com/en/mining/6?pool=CETFull details: https://www.coinex.com

In [24]:
str1 = message_list[1]

msg_str_stage1 = scrape.pullsubstr_begend(regex_start, regex_end, str1)
msg_str_stage2 = re.sub(re_cut, '', msg_str_stage1)
# msg_str_stage3 = re.sub(r'\s+','',msg_str_stage2).strip()
print(msg_str_stage2)

message='#ADX 👈https://www.binance.com/en/trade/ADX_BTCBuying Zone 230-250 👆Sell🤑 270-280🤑 280-310🚀 310-340💃 340-370 😱 370- 400 & aboveBullish over - 220',
	


In [47]:
regex_start = r'message=(["\'])(.*?)\1' # captures either single or double quote
regex_end = r"out=False"
re.search(regex_start, message_list[2]).span()[0]

151

In [46]:
print(message_list[3])

Message(
	id=8087,
	peer_id=PeerChannel(
		channel_id=1325133208
	),
	date=datetime.datetime(2024, 12, 11, 14, 35, 46, tzinfo=datetime.timezone.utc),
	message='🔼 🥰😇🥰🥰 \n\nhits an Impressive 202.57% APY! 👀\n\n📣CoinEx’s latest empowerment program is setting new records—this is the highest APY in 2 years! \n\n▶️ Don’t miss CoinEx’s 6th Launchpool Event: \n🎆Start with just 100 $CET \n🟥Dive into a massive 13,000,000 $HOLD pool 💰 \n✅Estimated APY could soar as high as 103%! 😱 \n\n▶️Claim your $CET now: https://www.coinex.com/en/mining/6?pool=CET\n\nFull details:\xa0 https://www.coinex.com/en/announcements/detail/32916644706964\n\n💥Ride the $CET wave : Earn rewards & benefit from its rising value! \nCoinEx’s buyback strategy uses 20% of daily trading fees to repurchase $CET, which is \nburned monthly until there’s no $CET left in circulation—creating a deflationary model. 🔥\n\nCurrent burn rate: 72.15%! 🔥\n\n✨✨✨✨✨✨✨✨\n\nUse My Referral link to Sign Up:\n\nhttps://www.coinex.com/register?rc=9q